# 네이버 Kkma 사용하여 summarize

In [15]:
import requests
import nltk
from bs4 import BeautifulSoup as bs
from gensim.summarization.summarizer import summarize
import string
from konlpy.tag import Kkma
from konlpy.tag import Hannanum

In [ ]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0010996363'

resp = requests.get(url)
soup = bs(resp.content)
content = soup.find('div',class_='_article_body_contents').text
content = content[72:-73]

In [73]:
def get_sentences(txt):
    kkma = Kkma()
    han = Hannanum()
    return kkma.sentences(txt)

def get_words(txt):
    kkma = Kkma()
    return kkma.morphs(txt)

def get_keywords(word_list,min_ratio =0.001,max_ratio = 0.5):
    assert(min_ratio < 1 and max_ratio < 1)
    
    count_dict = {}
    for word in word_list:
        count_dict.setdefault(word,0)
        count_dict[word] += 1
        
    keywords = set()
#     print(len(count_dict))
    for word,cnt in count_dict.items():
        word_percentage = count_dict[word]/len(word_list)
#         print(word_percentage)
        if word_percentage <= max_ratio and word_percentage >=min_ratio:
#             print("if")
            keywords.add(word)
    return keywords

def get_sentence_weight(sentence,keywords):
    kkma = Kkma()
    sen_list = kkma.morphs(sentence)
    window_start = 0; window_end = -1;
    
    for i in range(len(sen_list)):
        if sen_list[i] in keywords:
            window_start = i
            break
    
    for i in range(len(sen_list)-1,0,-1):
        if sen_list[i] in keywords:
            window_end = i
            break
    
    if window_start > window_end:
        return 0
    
    window_size = window_end - window_start +1
    
    keywords_cnt = 0
    for w in sen_list:
        if w in keywords:
            keywords_cnt += 1
    
    return keywords_cnt*keywords_cnt *1.0/window_size

def summarize(content,max_no_of_sentences = 10):
    txt = content
    word_list = get_words(txt)
    
    keywords = get_keywords(word_list)
    
#     print(keywords)
    
    sentence_list = get_sentences(txt)
    sentence_weight = []
    
    for sen in sentence_list:
        sentence_weight.append((get_sentence_weight(sen,keywords),sen))
        
    sentence_weight.sort(reverse = True)
    
    ret_list =[]
    ret_cnt = min(max_no_of_sentences,len(sentence_list))
#     print(sentence_weight)
    for i in range(ret_cnt):
        ret_list.append(str(sentence_weight[i][1]))
        
    return ret_list

In [77]:
summarize(content,1)

['왼쪽부터 이 석수 기획조정실장, 서 동구 1차 장, 서훈 원장, 김상 균 2차 장. 2019.8.1 toadboy@yna .co .kr 북한이 지난달 31일 발사한 발사체에 대해서는 " 비행거리 250여 ㎞, 고도는 30여 ㎞ 로 판단되고, 비행 제원의 특성이 신형 단거리 탄도 미사일과 유사하지만, 북한이 신형 대구경 조종 방사 포라고 주장하고 있어 추가 분석 중" 이라고 말했다.']

In [2]:
from collections import defaultdict

word2id = defaultdict((lambda:0))

In [8]:
defaultdict?